In [1]:
#im skipping leap days currently in multiple places
import sys
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import seaborn
import random
import json
import pickle
import time
import copy

import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [12]:
#ERCOT data
subregions = ['COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST', 'ERCOT']
other_subregions = ['COAST', 'EAST', 'FWEST', 'NORTH', 'NCENT', 'SOUTH', 'SCENT', 'WEST', 'ERCOT']


lds = datetime.datetime(year=2016, month=2, day=29, hour=0, minute=0) #lines 1415 to 1439
lde = datetime.datetime(year=2016, month=2, day=29, hour=23, minute=0)

path = "/home/chase/projects/peakload/data/ercot/"
ercot2017 = pd.read_excel(path + "native_Load_2017.xlsx") #different subregions Index(['Hour Ending', 'COAST', 'EAST', 'FWEST', 'NORTH', 'NCENT', 'SOUTH', 'SCENT', 'WEST', 'ERCOT'],dtype='object')
ercot2016 = pd.read_excel(path + "native_Load_2016.xlsx") #leap year
#ercot2016.drop(ercot2016.index[1415:1439], inplace=True)

ercot2015 = pd.read_excel(path + "native_Load_2015.xls")
ercot2014 = pd.read_excel(path + "2014_ERCOT_Hourly_Load_Data.xls")
ercot2013 = pd.read_excel(path + "2013_ERCOT_Hourly_Load_Data.xls")
ercot2012 = pd.read_excel(path + "2012_ERCOT_Hourly_Load_Data.xls") #leap year
#ercot2012.drop(ercot2012.index[1415:1439], inplace=True)

ercot2011 = pd.read_excel(path + "2011_ERCOT_Hourly_Load_Data.xls")
ercot2010 = pd.read_excel(path + "2010_ERCOT_Hourly_Load_Data.xls")

#ercotdata = list(ercot2010['ERCOT']) + list(ercot2011['ERCOT']) + list(ercot2012['ERCOT']) + list(ercot2013['ERCOT']) + list(ercot2014['ERCOT']) + list(ercot2015['ERCOT']) + list(ercot2016['ERCOT']) + list(ercot2017['ERCOT'])
yearly_data = {}
yearly_data['ERCOT'] = [list(ercot2010['ERCOT']), list(ercot2011['ERCOT']), list(ercot2012['ERCOT']), list(ercot2013['ERCOT']), list(ercot2014['ERCOT']), list(ercot2015['ERCOT']), list(ercot2016['ERCOT']), list(ercot2017['ERCOT'])]

for region in subregions:
    yearly_data[region] = [list(ercot2010[region]), list(ercot2011[region]), list(ercot2012[region]), list(ercot2013[region]), list(ercot2014[region]), list(ercot2015[region]), list(ercot2016[region])]

for region in other_subregions:
    r = subregions[other_subregions.index(region)]
    yearly_data[r].append(list(ercot2017[region]))

In [5]:
startdate = datetime.datetime(year=2010, month=1, day=1, hour=0, minute=0, second=0)
curr = startdate
enddate = datetime.datetime(year=2018, month=1, day=1, hour=0, minute=0, second=0)

allhours = []

while curr < enddate:
    allhours.append(curr)
    curr += datetime.timedelta(hours=1)
    
trainhours = allhours[0:-8760] #all but last year, not a leap year
testhours = allhours[-8760:]
print(len(allhours))
print(len(trainhours))
print(len(testhours))

70128
61368
8760


In [5]:
#with eliminated leap day, for calculating average year model
#yearly_loads = np.zeros((8,8760))
#
#for i in range(len(yearly_data)):
#    yearly_loads[i,:] = np.array(yearly_data[i]['ERCOT'])

In [3]:
testhours

NameError: name 'testhours' is not defined

In [13]:
#for subregions
vectorized_subregions = {}
for region in subregions:
    vectorized_subregions[region] = {}
    train_data_vec = np.array(yearly_data[region][0]) - np.mean(yearly_data[region][0])
    means = [np.mean(yearly_data[region][0])]
    for i in range(1, len(yearly_data[region])-1):
        yr = yearly_data[region][i]
        m = np.nanmean(yr)
        means.append(m)
        train_data_vec = np.concatenate((train_data_vec, np.array(yr)-m), axis=0)
        
    vectorized_subregions[region]['train_data'] = train_data_vec

    train_data_labels = np.zeros((len(yearly_data[region][0]),))
    for i in range(len(yearly_data[region][0])):
        greater = np.sum([1 for k in yearly_data[region][0] if k >= yearly_data[region][0][i]])/float(len(yearly_data[region][0]))
        train_data_labels[i] = 1.0 - greater
    
    for i in range(1, len(yearly_data[region])-1):
        year_data = yearly_data[region][i]
        train_data_l = np.zeros((len(year_data),))
        for j in range(len(year_data)):
            greater = np.sum([1 for k in year_data if k <= year_data[j]])/float(len(year_data))
            train_data_l[j] = greater
        train_data_labels = np.concatenate((train_data_labels, train_data_l), axis=0)
            
    vectorized_subregions[region]['train_labels'] = train_data_labels
    
    vectorized_subregions[region]['means'] = means
        
    

In [14]:
len(vectorized_subregions['ERCOT']['train_data'])

61368

In [15]:
#use system-wide loadgrowth for all regions
for region in vectorized_subregions:
    A = np.vstack((np.array([0, 1, 2, 3, 4, 5, 6]), np.array([1, 1, 1, 1, 1, 1, 1]))).T
    a, b = np.linalg.lstsq(A, np.array(vectorized_subregions[region]['means']))[0]
    print(a, b)
    projected_mean = a*7.0 + b
    vectorized_subregions[region]['projected_mean'] = projected_mean

202.21956698167793 10330.675211917747
1.1445290359099922 1389.0497308542554
157.5580701856505 1235.861525615093
-6.286314908476045 862.2773959771353
55.294436102682454 12689.296038363827
74.74464079097235 2906.584283652104
69.80779480723267 6095.32943668585
14.267152541100186 1046.8526042098674
568.7498755367494 36555.92622727587


/home/chase/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [16]:
for region in vectorized_subregions:
    test_data_vec = np.array(yearly_data[region][-1]) - vectorized_subregions[region]['projected_mean']

    test_data_labels = np.zeros((test_data_vec.shape[0],))        
    for i in range(test_data_vec.shape[0]):
        greater = np.sum([1 for k in test_data_vec if k >= test_data_vec[i]])/float(test_data_vec.shape[0])
        test_data_labels[i] = 1.0 - greater
        
    vectorized_subregions[region]['test_data'] = test_data_vec
    vectorized_subregions[region]['test_labels'] = test_data_labels

In [27]:
for region in vectorized_subregions:
    print(vectorized_subregions[region]['train_data'].shape, vectorized_subregions[region]['test_data'].shape, vectorized_subregions[region]['train_labels'].shape, vectorized_subregions[region]['test_labels'].shape)

(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)
(61368,) (8760,) (61368,) (8760,)


In [28]:
def itermean(oldmean, newval, num):
    if np.isnan(newval):
        out = oldmean
    else:
        out = oldmean + ((newval - oldmean)/float(num))
    return(out)

def itervar(oldvar, oldmean, newval, num):
    if np.isnan(newval):
        out = oldvar
    else:
        n = float(num)
        out = ((n - 1)*oldvar + (newval - oldmean)*(newval - oldmean))/n
    return(out)

In [29]:
all_loads = np.concatenate((vectorized_subregions['ERCOT']['train_data'], vectorized_subregions['ERCOT']['test_data']))
print(all_loads.shape)
print(len(allhours))
print(len(testhours))
print(len(trainhours))

(70128,)
70128
8760
61368


In [31]:
#do this for 'ERCOT' only


load_features = {}
seasons = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 2, 8: 2, 9: 2, 10: 3, 11: 3, 12: 3}

maxsofar = 0.0
meansofar = 0.0
varsofar = 0.0
year = 2010
for i in range(len(allhours)):
    load_features[allhours[i]] = {}
    loads_dayhour = []
    loads_hour = []
    
    currtime = allhours[i]
    curr_year = currtime.year
    if curr_year > year:
        year = curr_year
        maxsofar = 0.0
        meansofar = 0.0
        varsofar = 0.0
    if all_loads[i] > maxsofar:
        maxsofar = all_loads[i]
    varsofar = itervar(varsofar, meansofar, all_loads[i], i)
    meansofar = itermean(meansofar, all_loads[i], i)
    
    dt_back = currtime
    stop_1 = currtime - datetime.timedelta(days=30*4)
    stop_2 = currtime - datetime.timedelta(days=30*2)
    
    while dt_back > stop_1:
        if dt_back < allhours[0]:
            break
        if dt_back.month == 2 and dt_back.day == 29:
            pass
        else:
            #get loads over the last 2 months by matching day/hour
            if dt_back.hour == currtime.hour and dt_back.weekday() == currtime.weekday():
                loads_dayhour.append(all_loads[allhours.index(dt_back)])
            #get loads over the last 2 months by matching hour
            if dt_back > stop_2:
                if dt_back.hour == currtime.hour:
                    loads_hour.append(all_loads[allhours.index(dt_back)])
        dt_back -= datetime.timedelta(hours=24)
    
    dhhist = np.histogram(loads_dayhour, bins=7, range=(np.nanmin(loads_dayhour), np.nanmax(loads_dayhour)))[0]
    dayhourmax = np.nanmax(loads_dayhour)
    dayhourmin = np.nanmin(loads_dayhour)
    dayhourhist = (1.0/sum(dhhist)) * dhhist
    dayhourmean = np.nanmean(loads_dayhour)
    dayhourvar = np.nanvar(loads_dayhour)
    hourmax = np.nanmax(loads_hour)
    hourmin = np.nanmin(loads_hour)
    hhist = np.histogram(loads_hour, bins=7, range=(np.nanmin(loads_hour), np.nanmax(loads_hour)))[0]
    hourhist = (1.0/sum(hhist)) * hhist
    hourmean = np.nanmean(loads_hour)
    hourvar = np.nanvar(loads_hour)
    
    load_features[currtime]["dayhourmax"] = dayhourmax #1
    load_features[currtime]["dayhourmin"] = dayhourmin #1
    load_features[currtime]["dayhourhist"] = dayhourhist #7
    load_features[currtime]["dayhourmean"] = dayhourmean #1
    load_features[currtime]["dayhourvar"] = dayhourvar #1
    load_features[currtime]["hourmax"] = hourmax #1
    load_features[currtime]["hourmin"] = hourmin #1
    load_features[currtime]["hourhist"] = hourhist #7
    load_features[currtime]["hourmean"] = hourmean #1
    load_features[currtime]["hourvar"] = hourvar #1
    load_features[currtime]["maxloadsofar"] = maxsofar #1
    load_features[currtime]["meanloadsofar"] = meansofar #1
    load_features[currtime]["varsofar"] = varsofar #1
    load_features[currtime]["month"] = currtime.month #1
    load_features[currtime]["week"] = currtime.isocalendar()[1] #1
    load_features[currtime]["weekday"] = currtime.weekday() #1
    load_features[currtime]["season"] = seasons[currtime.month] #1

/home/chase/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: RuntimeWarning: divide by zero encountered in double_scalars
/home/chase/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in double_scalars
/home/chase/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars
/home/chase/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in double_scalars
/home/chase/applications/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:780: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/chase/applications/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:781: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


In [32]:
load_features_copy = copy.copy(load_features)

In [33]:
out_features = {}
for key in load_features_copy:
    t = str(int(time.mktime(key.timetuple())))
    out_features[t] = {}
    for kk in load_features_copy[key]:
        item = load_features_copy[key][kk]
        if type(item) == np.ndarray:
            out_features[t][kk] = item.tolist()
        else:
            out_features[t][kk] = item
        

with open("/home/chase/projects/peakload/data/ercot/load_features.json", 'w') as d:
    out = json.dumps(out_features)
    d.write(out)

In [7]:
#reload feature data
with open("/home/chase/projects/peakload/data/ercot/load_features.json", 'r') as d:
    load_features = json.load(d)

In [8]:
#feature construction
lookback = 24*7
numfeatures = 17

In [19]:
times = list(load_features.keys())
test_times = times[-8760:]

In [34]:
#training data input arrays, output_arrays 24 hours ahead
training_data_pairs = []

for i in range(len(trainhours)-lookback-24): #must offset for last 24 hours in dataset
    dt = trainhours[i]
    currtime = str(int(time.mktime(trainhours[i].timetuple())))
    data_point = np.zeros((lookback, numfeatures)) #current number of features
    label = np.zeros((24,1))
    for j in range(lookback):
        features = load_features[currtime]
        data_point[j,0] = vectorized_subregions['ERCOT']['train_data'][i+j]
        #data_point[j,1:8] = features["hourhist"]
        #data_point[j,8:15] = features["dayhourhist"]vectorized_subregions['ERCOT']['train_data'][i+j]
        data_point[j,1] = features["dayhourmax"]
        data_point[j,2] = features["dayhourmin"]
        data_point[j,3] = features["dayhourmean"]
        data_point[j,4] = features["dayhourvar"]
        data_point[j,5] = features["hourmax"]
        data_point[j,6] = features["hourmin"]
        data_point[j,7] = features["hourmean"]
        data_point[j,8] = features["hourvar"]
        data_point[j,9] = features["month"]
        data_point[j,10] = features["week"]
        data_point[j,11] = features["weekday"]
        data_point[j,12] = features["season"]
        data_point[j,13] = features["maxloadsofar"]
        data_point[j,14] = features["meanloadsofar"]
        data_point[j,15] = features["varsofar"]
        data_point[j,16] = float(dt.hour)/24.0
    for j in range(24):
        label[j,0] = train_data_labels[(i+24)+j]
    training_data_pairs.append((data_point, label))        

In [35]:
test_data_pairs = []
for i in range(len(testhours)-lookback-24):
    dt = testhours[i]
    currtime = str(int(time.mktime(testhours[i].timetuple())))
    data_point = np.zeros((lookback,numfeatures))
    label = np.zeros((24,1))
    for j in range(lookback):
        features = load_features[currtime]
        data_point[j,0] = vectorized_subregions['ERCOT']['test_data'][i+j]
        #data_point[j,1:8] = features["hourhist"]
        #data_point[j,8:15] = features["dayhourhist"]
        data_point[j,1] = features["dayhourmax"]
        data_point[j,2] = features["dayhourmin"]
        data_point[j,3] = features["dayhourmean"]
        data_point[j,4] = features["dayhourvar"]
        data_point[j,5] = features["hourmax"]
        data_point[j,6] = features["hourmin"]
        data_point[j,7] = features["hourmean"]
        data_point[j,8] = features["hourvar"]
        data_point[j,9] = features["month"]
        data_point[j,10] = features["week"]
        data_point[j,11] = features["weekday"]
        data_point[j,12] = features["season"]
        data_point[j,13] = features["maxloadsofar"]
        data_point[j,14] = features["meanloadsofar"]
        data_point[j,15] = features["varsofar"]
        data_point[j,16] = float(dt.hour)/24.0
    for j in range(24):
        label[j,0] = test_data_labels[(i+24)+j]
    test_data_pairs.append((data_point, label))
    
all_data_pairs = training_data_pairs + test_data_pairs

In [36]:
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_training_data_pairs.pck", 'wb') as d:
    pickle.dump(training_data_pairs, d)
    
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_test_data_pairs.pck", 'wb') as d:
    pickle.dump(test_data_pairs, d)

In [3]:
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_test_data_pairs.pck", 'rb') as d:
    test_data_pairs = pickle.load(d)
    
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_training_data_pairs.pck", 'rb') as d:
    training_data_pairs = pickle.load(d)

In [37]:
normcolumns = [i for i in range(17)]

#only get normalization values from training data
colmaxs = np.zeros((1,numfeatures))
colmins = np.zeros((1,numfeatures))
for pair in training_data_pairs:
    currmax = np.max(pair[0],axis=0)
    currmax = currmax[np.newaxis,:]
    currmin = np.min(pair[0],axis=0)
    currmin = currmin[np.newaxis,:]
    
    cmax = np.concatenate((colmaxs, currmax), axis=0)
    cmin = np.concatenate((colmins, currmin), axis=0)
    
    colmaxs = np.max(cmax, axis=0)[np.newaxis,:]
    colmins = np.min(cmin, axis=0)[np.newaxis,:]

In [38]:
normed_training_data_pairs = []
normed_test_data_pairs = []

for pair in training_data_pairs:
    normedarr = copy.copy(pair[0])
    for column in normcolumns:
        coldata = normedarr
        coldata = (1.0/(colmaxs[0,column] - colmins[0,column])) * (pair[0][:,column] - colmins[0,column])
        normedarr[:,column] = coldata.T
    
    normed_training_data_pairs.append((normedarr, pair[1]))

In [39]:
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_training_data_pairs.pck", 'wb') as d:
    pickle.dump(normed_training_data_pairs, d)

In [40]:
for pair in test_data_pairs:
    normedarr = copy.copy(pair[0])
    for column in normcolumns:
        coldata = normedarr
        coldata = (1.0/(colmaxs[0,column] - colmins[0,column])) * (pair[0][:,column] - colmins[0,column])
        normedarr[:,column] = coldata.T
    
    normed_test_data_pairs.append((normedarr, pair[1]))

In [41]:
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_test_data_pairs.pck", 'wb') as d:
    pickle.dump(normed_test_data_pairs, d)

In [6]:
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_training_data_pairs.pck", 'rb') as d:
    normed_training_data_pairs = pickle.load(d)
    
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_test_data_pairs.pck", 'rb') as d:
    normed_test_data_pairs = pickle.load(d)

In [7]:
normed_training_data_pairs[0][0].shape

(168, 17)

In [8]:
len(normed_test_data_pairs)

8568

In [9]:
#load normalized weather data
weatherdatapath = "/home/chase/projects/peakload/data/weather/ercot/major_cities/array_formatted/"

with open(weatherdatapath + "hourly_data_feature_vector.pck", 'rb') as d:
    hourly_weather = pickle.load(d)
    
with open(weatherdatapath + "daily_data_feature_vector.pck", 'rb') as d:
    daily_weather = pickle.load(d)

In [10]:
allweatherhours = list(hourly_weather.keys())
print(len(allhours))

train_weatherhours = allweatherhours[0:-8760]
test_weatherhours = allweatherhours[-8760:]
print(len(allweatherhours))

70128
70128


In [8]:
#training data pairs
weather_array_path = "/home/chase/projects/peakload/data/weather/ercot/major_cities/array_formatted/yearly_normed_arrays/"

weather_normed_training_data_pairs = []
hourly_keys = list(hourly_weather.keys())

for i in range(len(normed_training_data_pairs)):
    pair = normed_training_data_pairs[i]
    p1 = np.nan_to_num(pair[0]).flatten()
    dtplus = trainhours[i+1]
    day = datetime.datetime(year=dtplus.year, month=dtplus.month, day=dtplus.day, hour=0, minute=0, second=0)
    day_weather_vec = daily_weather[day] #get the forecast
    hourly_weather_vecs = []
    for j in range(24):
        hourly_weather_vecs.append(hourly_weather[hourly_keys[i+j+1]])
        
    wforecast = np.concatenate((np.asarray(hourly_weather_vecs).flatten(), day_weather_vec.flatten()), axis=0)
    wf = np.nan_to_num(wforecast)
    
    weather_normed_training_data_pairs.append( (np.concatenate((p1, wf), axis=0), pair[1].flatten()) )

In [9]:
#write the normalized weather data pairs
i = 0
for pair in weather_normed_training_data_pairs:
    with open("/home/chase/projects/peakload/data/ercot/weather_pairs/training/pair_" + str(i) + ".pck", 'wb') as d:
        pickle.dump(pair, d)
    i += 1

In [11]:
#test data pairs
weather_normed_test_data_pairs = []

for i in range(len(normed_test_data_pairs)):
    pair = normed_test_data_pairs[i]
    p1 = np.nan_to_num(pair[0]).flatten()
    dtplus = testhours[i+1]
    day = datetime.datetime(year=dtplus.year, month=dtplus.month, day=dtplus.day, hour=0, minute=0, second=0)
    day_weather_vec = daily_weather[day] #get the forecast
    hourly_weather_vecs = []
    for j in range(24):
        hourly_weather_vecs.append(hourly_weather[test_weatherhours[i+j+1]])
        
    
    wforecast = np.concatenate((np.asarray(hourly_weather_vecs).flatten(), day_weather_vec.flatten()), axis=0)
    wf = np.nan_to_num(wforecast)
    
    weather_normed_test_data_pairs.append( (np.concatenate((p1, wf), axis=0), pair[1].flatten()) )

In [12]:
#write the normalized weather data pairs
i = 0
for pair in weather_normed_test_data_pairs:
    with open("/home/chase/projects/peakload/data/ercot/weather_pairs/test/pair_" + str(i) + ".pck", 'wb') as d:
        pickle.dump(pair, d)
    i += 1

17030

In [17]:
datadir = "/home/chase/projects/peakload/data/ercot/weather_pairs/"
trainfiles = os.listdir(datadir + "training/")

weather_normed_training_data_pairs = []
for f in trainfiles:
    with open(datadir + "training/" + f, 'rb') as d:
        weather_normed_training_data_pairs.append(pickle.load(d))

In [19]:
print(len(weather_normed_training_data_pairs))

61177


In [25]:
for i in range(len(weather_normed_training_data_pairs)):
    pair = weather_normed_training_data_pairs[i]
    if np.isnan(pair[0]).any():
        print("fail")
        for j in range(len(pair[0])):
            if np.isnan(pair[0][j]):
                pair[0][j] = 0.0
        weather_normed_training_data_pairs[i] = (pair[0], pair[1])

In [2]:
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_test_data_pairs.pck", 'rb') as d:
    normed_test_data_pairs = pickle.load(d)
    
with open("/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_training_data_pairs.pck", 'rb') as d:
    normed_training_data_pairs = pickle.load(d)

In [5]:
len(normed_test_data_pairs) + 168 + 24

8760

In [26]:
#shuffle training/test data pairs, split out validation

random.shuffle(weather_normed_training_data_pairs)
training_pairs = weather_normed_training_data_pairs[0:int(0.9*float(len(weather_normed_training_data_pairs)))]
val_pairs = weather_normed_training_data_pairs[int(0.9*float(len(weather_normed_training_data_pairs))):]

random.shuffle(weather_normed_test_data_pairs)

In [27]:
weather_normed_training_data_pairs[0][0].shape

(17030,)

In [3]:
class linear_net(nn.Module):
    def __init__(self):
        super(linear_net, self).__init__()
        self.D_in = training_pairs[0][0].shape[0]
        self.H1 = 10000 #increased hidden size
        #can try additional hidden layer
        self.H2 = 5000
        self.H3 = 500
        self.D_out = 24
        self.l1 = nn.Linear(self.D_in, self.H1)
        self.l2 = nn.Linear(self.H1, self.H2)
        self.l3 = nn.Linear(self.H2, self.H3)
        self.l4 = nn.Linear(self.H3, self.D_out)
    
    def forward(self, x):
        x = F.sigmoid(self.l1(x))
        x = F.tanh(self.l2(x))
        x = F.sigmoid(self.l3(x))
        x = self.l4(x)
        return(x)

In [22]:
#write parameter objects to pck files

call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/single_hidden/single_hidden.py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_training_data_pairs.pck"
test_data_path = "/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_test_data_pairs.pck"
model_name = "single_sigmoid"

epochs = 100
lookback = 168 #one week in hours
losses = ["L1", "EW"]
hidden_1 = [250, 500, 1000]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/single_hidden/train_and_test.sh", 'w')

for loss in losses:
    for h in hidden_1:
        params = {'LOOKBACK': lookback, 'HIDDEN_1': h, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'file'}
        parampath = "/home/chase/projects/peakload/src/python/nets/single_hidden/" + loss + "loss_h1_" + str(h) + "_params.pck"
        with open(parampath, 'wb') as p:
            pickle.dump(params, p)
        shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
        train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [25]:
#write parameter objects to pck files

call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/tanh_hidden/tanh_hidden.py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_training_data_pairs.pck"
test_data_path = "/home/chase/projects/peakload/data/ercot/formatted/load_features_normed_test_data_pairs.pck"
model_name = "tanh_sigmoid"

epochs = 100
lookback = 168 #one week in hours
losses = ["EW"]
hidden_1 = [250, 500, 1000, 1500]
hidden_2 = [250, 500, 1000, 1500]
hidden_3 = [250, 500, 1000, 1500]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/tanh_hidden/train_and_test.sh", 'w')

for loss in losses:
    for i in range(len(hidden_2)):
        for j in range(i, len(hidden_1)):
            h1 = hidden_1[j]
            h2 = hidden_2[i]
            h3 = hidden_3[i]
            params = {'LOOKBACK': lookback, 'HIDDEN_1': h1, 'HIDDEN_2': h2, 'HIDDEN_3': h3, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'file'}
            parampath = "/home/chase/projects/peakload/src/python/nets/tanh_hidden/" + loss + "loss_h1_" + str(h1) + "_h2_" + str(h2) + "_h3_" + str(h3) + "_params.pck"
            with open(parampath, 'wb') as p:
                pickle.dump(params, p)
            shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
            train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [5]:
#write parameter objects to pck files

call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/tanh_hidden/tanh_hidden.py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/training"
test_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/test"
model_name = "tanh_sigmoid_weather"

epochs = 100
lookback = 168 #one week in hours
losses = ["EW", "L1"]
hidden_1 = [250, 500, 1000, 1500, 2500, 5000, 7500, 10000, 20000]
hidden_2 = [250, 500, 1000, 1500, 2500, 5000, 7500]
hidden_3 = [50, 100, 250, 500, 1000, 1500, 2500, 5000]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/tanh_hidden/train_and_test.sh", 'w')

for loss in losses:
    for i in range(len(hidden_2)):
        for j in range(i, len(hidden_1)):
            h1 = hidden_1[j]
            h2 = hidden_2[i]
            h3 = hidden_3[i]
            params = {'LOOKBACK': lookback, 'HIDDEN_1': h1, 'HIDDEN_2': h2, 'HIDDEN_3': h3, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'directory'}
            parampath = "/home/chase/projects/peakload/src/python/nets/tanh_hidden/" + loss + "loss_h1_" + str(h1) + "_h2_" + str(h2) + "_h3_" + str(h3) + "_params.pck"
            with open(parampath, 'wb') as p:
                pickle.dump(params, p)
            shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
            train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [ ]:
#write parameter objects to pck files

call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/tanh_hidden/tanh_hidden.py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/training"
test_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/test"
model_name = "tanh_sigmoid_weather"

epochs = 100
lookback = 168 #one week in hours
losses = ["EW", "L1"]
hidden_1 = [250, 500, 1000, 1500, 2500, 5000, 7500, 10000, 20000]
hidden_2 = [250, 500, 1000, 1500, 2500, 5000, 7500]
hidden_3 = [50, 100, 250, 500, 1000, 1500, 2500, 5000]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/tanh_hidden/train_and_test.sh", 'w')

for loss in losses:
    for i in range(len(hidden_2)):
        for j in range(i, len(hidden_1)):
            h1 = hidden_1[j]
            h2 = hidden_2[i]
            h3 = hidden_3[i]
            params = {'LOOKBACK': lookback, 'HIDDEN_1': h1, 'HIDDEN_2': h2, 'HIDDEN_3': h3, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'directory'}
            parampath = "/home/chase/projects/peakload/src/python/nets/tanh_hidden/" + loss + "loss_h1_" + str(h1) + "_h2_" + str(h2) + "_h3_" + str(h3) + "_params.pck"
            with open(parampath, 'wb') as p:
                pickle.dump(params, p)
            shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
            train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [5]:
#write parameter objects to pck files

model_name = "tanh_dropout"
call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/" + model_name + "/" + model_name + ".py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/training"
test_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/test"

epochs = 100
lookback = 168 #one week in hours
losses = ["EW", "L1"]
hidden_1 = [1500, 2500, 5000, 7500, 10000, 20000]
hidden_2 = [250, 1000, 2500, 5000]
hidden_3 = [25, 100, 200, 250]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/" + model_name + "/train_and_test.sh", 'w')

for loss in losses:
    for i in range(len(hidden_2)):
        for j in range(i, len(hidden_1)):
            h1 = hidden_1[j]
            h2 = hidden_2[i]
            h3 = hidden_3[i]
            params = {'LOOKBACK': lookback, 'HIDDEN_1': h1, 'HIDDEN_2': h2, 'HIDDEN_3': h3, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'directory'}
            parampath = "/home/chase/projects/peakload/src/python/nets/" + model_name + "/" + loss + "loss_h1_" + str(h1) + "_h2_" + str(h2) + "_h3_" + str(h3) + "_params.pck"
            with open(parampath, 'wb') as p:
                pickle.dump(params, p)
            shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
            train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [ ]:
#write parameter objects to pck files

model_name = "tanh_deep"
call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/" + model_name + "/" + model_name + ".py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/training"
test_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/test"

epochs = 100
lookback = 168 #one week in hours
losses = ["EW", "L1"]
hidden_1 = [1500, 2500, 5000, 7500, 10000, 20000]
hidden_2 = [1000, 1500, 2500, 5000, 7500]
hidden_3 = [50, 500, 1000]
hidden_4 = [50, 500, 1000]
hidden_5 = [50, 500, 1000]
hidden_6 = [50, 100]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/" + model_name + "/train_and_test.sh", 'w')

for loss in losses:
    for i in range(len(hidden_2)):
        for j in range(i, len(hidden_1)):
            h1 = hidden_1[j]
            h2 = hidden_2[i]
            h3 = hidden_3[i]
            h4 = hidden_4[i]
            h5 = hidden_5[i]
            h6 = hidden_6[i]
            params = {'LOOKBACK': lookback, 'HIDDEN_1': h1, 'HIDDEN_2': h2, 'HIDDEN_3': h3, 'HIDDEN_4': h4, 'HIDDEN_5': h5, 'HIDDEN_6': h6, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'directory'}
            parampath = "/home/chase/projects/peakload/src/python/nets/" + model_name + "/" + loss + "loss_h1_" + str(h1) + "_h2_" + str(h2) + "_h3_" + str(h3) + "_h4_" + str(h4) + "_h5_" + str(h5) + "_h6_" + str(h6) + "_params.pck"
            with open(parampath, 'wb') as p:
                pickle.dump(params, p)
            shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
            train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [ ]:
#write parameter objects to pck files

model_name = "tanh_deep_dropout"
call = "python /home/chase/projects/peakload/src/python/train.py"
net =  "/home/chase/projects/peakload/src/python/nets/" + model_name + "/" + model_name + ".py"
#paramfile = "/home/chase/projects/peakload/src/python/nets/single_hidden/EWloss_h1_250_params.pck"
train_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/training"
test_data_path = "/home/chase/projects/peakload/data/ercot/weather_pairs/test"

epochs = 100
lookback = 168 #one week in hours
losses = ["EW", "L1"]
hidden_1 = [1500, 2500, 5000, 7500, 10000, 20000]
hidden_2 = [1000, 1500, 2500, 5000, 7500]
hidden_3 = [50, 500, 1000]
hidden_4 = [50, 500, 1000]
hidden_5 = [50, 500, 1000]
hidden_6 = [50, 100]

train_test_file = open("/home/chase/projects/peakload/src/python/nets/" + model_name + "/train_and_test.sh", 'w')

for loss in losses:
    for i in range(len(hidden_2)):
        for j in range(i, len(hidden_1)):
            h1 = hidden_1[j]
            h2 = hidden_2[i]
            h3 = hidden_3[i]
            h4 = hidden_4[i]
            h5 = hidden_5[i]
            h6 = hidden_6[i]
            params = {'LOOKBACK': lookback, 'HIDDEN_1': h1, 'HIDDEN_2': h2, 'HIDDEN_3': h3, 'HIDDEN_4': h4, 'HIDDEN_5': h5, 'HIDDEN_6': h6, 'LOSS': loss, 'EPOCHS': epochs, 'DATA': 'directory'}
            parampath = "/home/chase/projects/peakload/src/python/nets/" + model_name + "/" + loss + "loss_h1_" + str(h1) + "_h2_" + str(h2) + "_h3_" + str(h3) + "_h4_" + str(h4) + "_h5_" + str(h5) + "_h6_" + str(h6) + "_params.pck"
            with open(parampath, 'wb') as p:
                pickle.dump(params, p)
            shcmd = " ".join([call, net, parampath, train_data_path, test_data_path, model_name])
            train_test_file.write(shcmd + ";\n")
        
train_test_file.close()

In [ ]:
#write a bash file to train all

In [21]:
class l_linear_net(nn.Module):
    def __init__(self):
        super(linear_net, self).__init__()
        self.D_in = lookback * 28
        self.H1 = 250
        self.H2 = 150
        self.H3 = 50
        self.D_out = 24
        self.l1 = nn.Linear(self.D_in, self.H1)
        self.l2 = nn.Linear(self.H1, self.H2)
        self.l3 = nn.Linear(self.H2, self.H3)
        self.l4 = nn.Linear(self.H3, self.D_out)
    
    def forward(self, x):
        x = F.sigmoid(self.l1(F.dropout(x)))
        x = F.sigmoid(self.l2(x))
        x = F.softmax(self.l3(x))
        x = self.l4(x)
        return(x)

In [29]:
def batch_sample_list(datalist, batchsize):
    remainder = len(datalist) % batchsize
    diff = batchsize - remainder
    tail = datalist[-diff:] + datalist[0:remainder]
    out = [ datalist[i*batchsize:(i+1)*batchsize] for i in range(int(float(len(datalist))/float(batchsize)))]
    out = out + [tail]
    return(out)

def torch_reshape_data(databatch):
    inputs = []
    labels = []
    for sample in databatch:
        inputs.append(sample[0].flatten())
        labels.append(sample[1].flatten())
    return(torch.Tensor(np.asarray(inputs)), torch.Tensor(np.asarray(labels)))

In [23]:
any(np.array([1,2,3]) > 2)

True

In [30]:
def batched_model_score(net_obj, datalist, thresh=0.85, batchsize=100):
    recs = []
    precs = []
    thresh_losses_low = []
    thresh_losses_high = []
    set_losses = []
    
    #batchdata
    batches = batch_sample_list(datalist, batchsize)
    
    #do prediction on datalist
    for i, data, in enumerate(batches):
        inputs, labels = torch_reshape_data(data)
        labels = Variable(labels.cuda())
        inputs = Variable(inputs.cuda())
        outputs = net_obj(inputs)
        loss = criterion(outputs, labels)
        set_losses.append(loss.data[0])
        
        labels = labels.data.cpu().numpy()
        outputs = outputs.data.cpu().numpy()
        
        for i in range(outputs.shape[0]):
            l = labels[i,:]
            o = outputs[i,:]
            
            if any(l > thresh):
                rec = binary_rec(l, o, thresh)
                recs.append(rec)
            if any(o > thresh):
                prec = binary_prec(l, o, thresh)
                precs.append(prec)
                
            loss_low = thresh_loss(l, o, thresh)
            loss_high = thresh_loss(l, o, 0.95)
            
            thresh_losses_low.append(np.mean(loss_low))
            thresh_losses_high.append(np.mean(loss_high))
    
    if len(recs) < 1:
        r = np.nan
    else:
        r = np.mean(recs)
    if len(precs) < 1:
        p = np.nan
    else:
        p = np.mean(precs)
    if np.isnan(r) or np.isnan(p):
        batch_f1 = np.nan
    else:
        batch_f1 = f1_score(r, p)
    
    low_thresh_mean_loss = np.nanmean(thresh_losses_low)
    high_thresh_mean_loss = np.nanmean(thresh_losses_high)
    mean_set_loss = np.mean(set_losses)
    
    ret = {"batch_recall": r, "batch_precision": p, "batch_f1": batch_f1, "low_thresh_mean_loss": low_thresh_mean_loss, "high_thresh_mean_loss": high_thresh_mean_loss, "mean_set_loss": mean_set_loss}          
    return(ret)

def print_model_scores(scores, name):
    print("====" + name + "====")
    for k in scores:
        print(k + ": " + str(scores[k]))
    print("=================\n")
    
    
def binary_rec(true, pred, thresh=0.85):
    true_pos = 0
    pred_pos = 0
    for i in range(len(true)):
        val = true[i]
        if val >= thresh:
            true_pos += 1
            pred_val = pred[i]
            if pred_val >= thresh:
                pred_pos += 1
    if true_pos == 0:
        return(0)
    else:
        return(float(pred_pos)/float(true_pos))
    
def binary_prec(true, pred, thresh=0.85):
    pos = 0
    true_pos = 0
    for i in range(len(pred)):
        val = pred[i]
        if val >= thresh:
            pos += 1
            comp_val = true[i]
            if comp_val >= thresh:
                true_pos += 1
    if pos == 0:
        return(0)
    else:
        return(float(true_pos)/float(pos))

def thresh_loss(true, pred, thresh=0.85):
    out = []
    for i in range(len(true)):
        val = true[i]
        if val >= thresh:
            loss = np.abs(val - pred[i])
            out.append(loss)
    return(out)

def f1_score(rec, prec):
    return(2.0/((1/rec) + (1/prec)))

def set_loss(net_obj, data):
    inputs, labels = torch_reshape_data(data)
    labels = Variable(labels.cuda())
    outputs = net_obj(Variable(inputs.cuda()))
    loss = criterion(outputs, labels)
    return(loss.data[0])

In [31]:
def model_score(net_obj, datalist, threshes=[0.85, 0.95, 0.99, 0.999], batchsize=100):
    #model scoring is on L1 loss
    recs = {}
    precs = {}
    thresh_losses = {}
    for t in threshes:
        thresh_losses[t] = []
        recs[t] = []
        precs[t] = []
    set_losses = []
    
    #for also recording weighted loss function later
    criterion_l1 = nn.L1Loss()
    
    #do prediction on datalist
    inputs, labels = torch_reshape_data(datalist)
    labels = Variable(labels.cuda())
    inputs = Variable(inputs.cuda())
    outputs = net_obj(inputs)
    loss = criterion_l1(outputs, labels)
    set_losses.append(loss.data[0])
    
    labels = labels.data.cpu().numpy()
    outputs = outputs.data.cpu().numpy()
    
    for i in range(outputs.shape[0]):
        l = labels[i,:]
        o = outputs[i,:]
        
        for t in threshes:
            if any(l > t):
                rec = binary_rec(l, o, t)
                recs[t].append(rec)
            if any(o > t):
                prec = binary_prec(l, o, t)
                precs[t].append(prec)
  
            tl = thresh_loss(l, o, t)
            thresh_losses[t].append(np.nanmean(tl))
    
    for t in thresh_losses:
        thresh_losses[t] = np.nanmean(thresh_losses[t])
        if len(recs[t]) < 1:
            recs[t] = np.nan
        else:
            recs[t] = np.nanmean(recs[t])
        if len(precs[t]) < 1:
            precs[t] = np.nan
        else:
            precs[t] = np.nanmean(precs[t])

    mean_set_loss = np.nanmean(set_losses)
    
    ret = {"batch_recall": recs, "batch_precision": precs, "thresh_losses": thresh_losses, "mean_set_loss": mean_set_loss}          
    return(ret)

In [25]:
import math

torch.exp(torch.Tensor([0, math.log(2)])) * torch.exp(torch.Tensor([0, math.log(2)]))


 1
 4
[torch.FloatTensor of size 2]

In [32]:
def expw_mae_loss(input, target, base=10.0):
    base = Variable(torch.Tensor([base])).type_as(target)
    return(( (input - target).abs() * torch.pow(base, target) ).sum() / input.data.nelement() )

In [33]:
def train(net_obj, loss_fnc, opt_fnc, traindata, valdata, batchsize=100, epochs=50):  
    train_batches = batch_sample_list(traindata, batchsize)
    num_batches = len(train_batches)
    tenth = int(num_batches/10)
    
    training_epoch_loss = []
    train_epoch_set_loss = []
    val_epoch_set_loss = []
    
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_batches):
            inputs, labels = torch_reshape_data(data)

            # wrap them in Variable
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda()) #they need to be .cuda() with each training epoch

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net_obj(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.data[0]

            #training loss
            #print("epoch " + str(epoch) + " ,iter " + str(i) + ": " + str(running_loss))
            #training_epoch_loss.append(running_loss)
        
        print("==epoch " + str(epoch) + "==")
        train_samp = random.randint(0,num_batches-1)
        train_set_loss = set_loss(net_obj, train_batches[train_samp])
        train_epoch_set_loss.append(train_set_loss)
        print("training set sampled loss: " + str(train_set_loss))
        val_set_loss = set_loss(net_obj, valdata)
        val_epoch_set_loss.append(val_set_loss)
        print("validation set loss: " + str(val_set_loss))
       
    #sampled epoch set loss
    print("\n")
    print("Model results")
    
    train_model_scores = batched_model_score(net_obj, traindata)
    print_model_scores(train_model_scores, "training data scores")
    
    val_model_scores = batched_model_score(net_obj, valdata)
    print_model_scores(val_model_scores, "validation data scores")
                
    return(training_epoch_loss, train_epoch_set_loss, val_epoch_set_loss)

In [ ]:
#switch nans to -1's

In [34]:
batch_size = 100 #batch size

net = linear_net().cuda()

criterion = expw_mae_loss #nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_losses, train_accs, test_accs = train(net, criterion, optimizer, training_pairs, val_pairs)

==epoch 0==
training set sampled loss: 0.8030710816383362


In [60]:
test_model_scores = model_score(net, weather_normed_test_data_pairs)
print_model_scores(test_model_scores, "test data scores")

/home/chase/applications/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:39: RuntimeWarning: Mean of empty slice


====test data scores====
batch_recall: {0.85: 0.8184080665931602, 0.95: 0.9420365780671124, 0.99: 0.9969432314410481, 0.999: 1.0}
batch_precision: {0.85: 0.8928108353067051, 0.95: 0.6117134316069153, 0.99: 0.15997180162558833, 0.999: 0.007426426426426425}
thresh_losses: {0.85: 0.042240818868496385, 0.95: 0.05805267103670435, 0.99: 0.08449693017515857, 0.999: 0.11889468113037005}
mean_set_loss: 0.1052764281630516



In [ ]:
def l1_loss(input, target, size_average=True):
    return _functions.thnn.L1Loss.apply(input, target, size_average)ls


class L1Loss(_Loss):
    r"""Creates a criterion that measures the mean absolute value of the
    element-wise difference between input `x` and target `y`:

    :math:`{loss}(x, y)  = 1/n \sum |x_i - y_i|`

    `x` and `y` arbitrary shapes with a total of `n` elements each.

    The sum operation still operates over all the elements, and divides by `n`.

    The division by `n` can be avoided if one sets the constructor argument
    `size_average=False`.

    Args:
        size_average (bool, optional): By default, the losses are averaged
           over observations for each minibatch. However, if the field
           size_average is set to False, the losses are instead summed for
           each minibatch. Default: True

    Shape:
        - Input: :math:`(N, *)` where `*` means, any number of additional
          dimensions
        - Target: :math:`(N, *)`, same shape as the input

    Examples::

        >>> loss = nn.L1Loss()
        >>> input = autograd.Variable(torch.randn(3, 5), requires_grad=True)
        >>> target = autograd.Variable(torch.randn(3, 5))
        >>> output = loss(input, target)
        >>> output.backward()
    """
    def forward(self, input, target):
        _assert_no_grad(target)
        return F.l1_loss(input, target, size_average=self.size_average)

0.8888888888888888


array([ 0.22591324,  0.28630137,  0.3872146 ,  0.45502284,  0.48664382,
        0.47203195,  0.43481734,  0.38972601,  0.36004567,  0.34406394,
        0.33755708,  0.36883563,  0.43584475,  0.45285389,  0.40958905,
        0.3303653 ,  0.23504566,  0.16575342,  0.11312786,  0.08070777,
        0.0609589 ,  0.0553653 ,  0.05947489,  0.08413242], dtype=float32)

array([ 0.27877405,  0.29841518,  0.31782278,  0.3170574 ,  0.30463257,
        0.30868047,  0.3124792 ,  0.3071259 ,  0.31804854,  0.31793466,
        0.31255594,  0.30287418,  0.30706862,  0.27466702,  0.26765308,
        0.23752765,  0.21457636,  0.19224998,  0.19455633,  0.19908968,
        0.2263395 ,  0.23829129,  0.25208017,  0.26730546], dtype=float32)

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [323]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.c1 = nn.Conv1d(input_size, hidden_size, 2)
        self.p1 = nn.AvgPool1d(2)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 1)
        self.p2 = nn.AvgPool1d(2)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=0.01)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, inputs, hidden):
        batch_size = inputs.size(1)
        
        # Turn (seq_len x batch_size x input_size) into (batch_size x input_size x seq_len) for CNN
        inputs = inputs.transpose(0, 1).transpose(1, 2)

        # Run through Conv1d and Pool1d layers
        c = self.c1(inputs)
        p = self.p1(c)
        c = self.c2(p)
        p = self.p2(c)

        # Turn (batch_size x hidden_size x seq_len) back into (seq_len x batch_size x hidden_size) for RNN
        p = p.transpose(1, 2).transpose(0, 1)
        
        p = F.tanh(p)
        output, hidden = self.gru(p, hidden)
        conv_seq_len = output.size(0)
        output = output.view(conv_seq_len * batch_size, self.hidden_size) # Treating (conv_seq_len x batch_size) as batch_size for linear layer
        output = F.tanh(self.out(output))
        output = output.view(conv_seq_len, -1, self.output_size)
        return output, hidden

input_size = 20
hidden_size = 50
output_size = 7
batch_size = 5
n_layers = 2
seq_len = 15

rnn = RNN(input_size, hidden_size, output_size, n_layers=n_layers)

inputs = Variable(torch.rand(seq_len, batch_size, input_size)) # seq_len x batch_size x 
outputs, hidden = rnn(inputs, None)
print('outputs', outputs.size()) # conv_seq_len x batch_size x output_size
print('hidden', hidden.size()) # n_layers x batch_size x hidden_size

outputs torch.Size([3, 5, 7])
hidden torch.Size([2, 5, 50])


In [329]:
inputs.data.shape

torch.Size([15, 5, 20])